<a href="https://colab.research.google.com/github/YordanIg/learning-ML/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perform some MNIST classifications with a fully connected FFN.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Use CUDA if available.
if torch.cuda.is_available():
    print("CUDA is available! Using GPU.")
    device = torch.device("cuda")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}") # Prints the name of the first GPU
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available! Using GPU.
GPU Name: Tesla T4


In [ ]:
# Load the training and testing datasets, transforming them to tensors.
to_tensor = transforms.ToTensor()
trainset  = torchvision.datasets.MNIST(root="./data", train=True,
                                       download=True, transform=to_tensor)
testset   = torchvision.datasets.MNIST(root="./data", train=False,
                                       download=True, transform=to_tensor)

In [ ]:
# Set up the data loaders for the training and test sets. Training loader should
# shuffle the data so the model doesn't memorise their order, while testing
# loader doesn't need to do that.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader  = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
# Set up the basic FFN to perform classification.
class FFN(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(28*28, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)
  def forward(self, x):
    x = x.view(-1, 28**2)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)  # Do not apply ReLU as will use CELoss.
    return x

In [ ]:
model = FFN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 8
for epoch in range(num_epochs):
  running_loss = 0
  for i, data in enumerate(trainloader):
    images, labels = data
    optimizer.zero_grad()              # Zeros the model's parameter gradients.
    outputs = model(images)            # Forward pass.
    loss = criterion(outputs, labels)  # Evaluate loss function.
    loss.backward()                    # Compute the parameter gradients.
    optimizer.step()                   # Use gradients to take a step.
    running_loss += loss.item()
    # Every 200 batches, print the average loss.
    if i%200==0 and i!=0:
      print(f"Epoch {epoch+1}, Batch {i+1}, loss {running_loss/200:0.3f}")
      running_loss = 0
print("Finished training.")

Epoch 1, Batch 1, loss 0.012
Epoch 1, Batch 201, loss 0.694
Epoch 1, Batch 401, loss 0.294
Epoch 1, Batch 601, loss 0.236
Epoch 1, Batch 801, loss 0.207
Epoch 2, Batch 1, loss 0.001
Epoch 2, Batch 201, loss 0.162
Epoch 2, Batch 401, loss 0.144
Epoch 2, Batch 601, loss 0.140
Epoch 2, Batch 801, loss 0.117
Epoch 3, Batch 1, loss 0.000
Epoch 3, Batch 201, loss 0.102
Epoch 3, Batch 401, loss 0.099
Epoch 3, Batch 601, loss 0.097
Epoch 3, Batch 801, loss 0.091
Epoch 4, Batch 1, loss 0.000
Epoch 4, Batch 201, loss 0.075
Epoch 4, Batch 401, loss 0.073
Epoch 4, Batch 601, loss 0.081
Epoch 4, Batch 801, loss 0.067
Epoch 5, Batch 1, loss 0.000
Epoch 5, Batch 201, loss 0.057
Epoch 5, Batch 401, loss 0.055
Epoch 5, Batch 601, loss 0.057
Epoch 5, Batch 801, loss 0.056
Epoch 6, Batch 1, loss 0.000
Epoch 6, Batch 201, loss 0.045
Epoch 6, Batch 401, loss 0.039
Epoch 6, Batch 601, loss 0.053
Epoch 6, Batch 801, loss 0.040
Epoch 7, Batch 1, loss 0.000
Epoch 7, Batch 201, loss 0.036
Epoch 7, Batch 401, lo

In [ ]:
# Evaluate the model.
correct = 0
total   = 0
with torch.no_grad():
  for data in testloader:
    images, labels = data
    outputs        = model(images)
    # Access only the output values (not their gradients) using outputs.data.
    # torch.max returns (value, index) - we only care about index.
    _, predicted   = torch.max(outputs.data, 1)
    total += labels.size(0)
    for pred, lab in zip(predicted, labels):
      if pred == lab:
        correct += 1

In [ ]:
accuracy = 100 * correct / total
print(f"Model accuracy is {accuracy}")

Model accuracy is 97.65
